# импорты

In [ ]:
! pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-install-ox0wluqp/sklearn-crfsuite_d2465201222a434f92a08ad745c213b7
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-install-ox0wluqp/sklearn-crfsuite_d2465201222a434f92a08ad745c213b7
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done


In [ ]:
! pip install stanza --quiet

In [ ]:
import pandas as pd

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [ ]:
import stanza

# Загрузка тренировочных данных

In [ ]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt

--2023-12-27 16:09:11--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228391 (223K) [text/plain]
Saving to: ‘train_aspects.txt.2’

train_aspects.txt.2 100%[===================>] 223.04K  --.-KB/s    in 0.009s  

2023-12-27 16:09:11 (23.9 MB/s) - ‘train_aspects.txt.2’ saved [228391/228391]

--2023-12-27 16:09:11--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting re

# Приведение тренировочного корпуса к формату BIO

In [ ]:
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize,pos,lemma')

INFO:stanza:Downloading default packages for language: ru (Russian) ...
INFO:stanza:File exists: /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
texts = pd.read_csv('train_reviews.txt', delimiter='\t', names = ['review_id', 'text'])

In [ ]:
texts.head()

,review_id,text
0,3976,"День 8-го марта прошёл, можно и итоги подвести..."
1,30808,Отмечали в этом ресторане день рождение на пер...
2,14031,Хочу поделиться своим впечатлением от посещени...
3,2495,Добрый день! Были вчера с друзьями в этом кафе...
4,38835,Отметили с мужем годовщину свадьбы 6 ноября в ...


In [ ]:
train_aspects = pd.read_csv('train_aspects.txt', delimiter='\t', names = ['review_id', 'category', 'mention', 'start', 'stop', 'sentiment'])

In [ ]:
train_aspects.head()

,review_id,category,mention,start,stop,sentiment
0,3976,Whole,ресторане,71,80,neutral
1,3976,Whole,ресторанах,198,208,neutral
2,3976,Whole,ресторане,256,265,neutral
3,3976,Service,Столик бронировали,267,285,neutral
4,3976,Service,администратор,322,335,positive


In [ ]:
bio_mentions = []
for idx, row in train_aspects.iterrows(): #меняем разметку аспектов
  mentions = row.mention.split()
  if len(mentions) == 1:
    bio_mentions.append({'review_id': row.review_id, 'mention' : row.mention, 'category' : 'B-' + row.category})
  else:
    bio_mentions.append({'review_id': row.review_id, 'mention' : mentions[0], 'category' : 'B-' + row.category})
    for i in range (1, len(mentions)):
      bio_mentions.append({'review_id': row.review_id, 'mention' : mentions[i], 'category' : 'I-' + row.category})

In [ ]:
bio_sentences = []
for row in texts.itertuples(): #проходимся по всем тренировочным текстам и приводим их к нужному для обучения crf формату
    doc = nlp(row.text)
    for sentence in doc.sentences:
      s = []
      for token in sentence.words:
        s.append([row.review_id, 'O', token.text, row.text.index(token.text), row.text.index(token.text) + len(token.text), token.lemma, token.pos])
      bio_sentences.append(s)


In [ ]:
for m in bio_mentions: #добавляем разметку
    for sentence in bio_sentences:
      for token in sentence:
        if m['review_id'] == token[0] and m['mention'] == token[2]:
            token[1] = m['category']

# Извлекаем параметры

In [ ]:
# приводим данные к нужному виду для обучения crf
def word2features(sent, i):
    word = sent[i][2]
    postag = sent[i][-1]
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.isdigit()': word.isdigit(),
        'postag': postag
    }
    if i > 0:
        word1 = sent[i-1][2]
        postag1 = sent[i-1][-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][2]
        postag1 = sent[i+1][-1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for _, label, token, _, _, _, _ in sent]

In [ ]:
import copy

In [ ]:
train_sents = copy.deepcopy(bio_sentences)

In [ ]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

In [ ]:
import pickle

filename = 'crf.pkl'
pickle.dump(crf, open(filename, 'wb'))

# примерная оценка качества для модели, обученной на части данных

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(bio_sentences)

In [ ]:
X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in test]
y_test = [sent2labels(s) for s in test]

In [ ]:
toy_crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

toy_crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100)

In [ ]:
y_pred = toy_crf.predict(X_test)

In [ ]:
labels = list(toy_crf.classes_)


In [ ]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3, zero_division=0.0
))

              precision    recall  f1-score   support

           O      0.939     0.981     0.960      9920
      B-Food      0.782     0.648     0.709       505
      I-Food      0.686     0.433     0.531       323
  B-Interior      0.815     0.647     0.721       170
  I-Interior      0.524     0.289     0.373        38
     B-Price      0.857     0.600     0.706        30
     I-Price      0.000     0.000     0.000         5
   B-Service      0.837     0.594     0.695       320
   I-Service      0.667     0.309     0.423        97
     B-Whole      0.822     0.733     0.775       195
     I-Whole      0.467     0.389     0.424        18

    accuracy                          0.921     11621
   macro avg      0.672     0.511     0.574     11621
weighted avg      0.914     0.921     0.915     11621



Лучше всего модель научилась распознавать то, что не является аспектом (O); хуже всего удается выделить сущности, состоящие из нескольких токенов (I-).
Главная проблема модели - недобучение из-за дисбаланса классов: примеров с O достаточно, а остальных -- нет.